In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

# Compare two libraries

In [ ]:
path_output = Path(holo._PATH_OUTPUT).resolve()
paths = [
    "ps-test-uniform_2023-03-01_n200_s50_r100_f40_SHARE",
    "ps-test-normal_2023-03-01_n200_s50_r100_f40_SHARE",
]
fname = "sam_lib.hdf5"
files = []
for path in paths:
    file = path_output.joinpath(path, fname)
    files.append(file)
    print(file.name, file.exists())

In [ ]:
fig, axes = plt.subplots(figsize=[10, 5], ncols=2, sharey=True, sharex=True)
for ax, file in zip(axes, files):
    ax.set(xscale='log', yscale='log')
    ax.grid(True, alpha=0.25)
    temp = file.parts[-2].split("_")[0].split("-")[-1]
    ax.set_title(temp, fontsize=12)
    data = h5py.File(file, 'r')

    fobs = data['fobs'][()]
    gwb = data['gwb'][()]
    nsamp, _, nreal = gwb.shape
    for ii in range(nsamp):
        plot.draw_gwb(ax, fobs, gwb[ii])
    
plt.show()


In [ ]:
nbins = [5, 10, 15, 0]

fit_keys = ['fit_lamp', 'fit_plaw']

for file in files:
    label = file.parts[-2].split("_")[0].split("-")[-1]
    data = h5py.File(file, 'r')
    lamp = data['fit_lamp'][:, :, 1]
    plaw = data['fit_plaw'][:, :, 1]
    vals = [lamp.flatten(), plaw.flatten()]
    vals = [vv[::10] for vv in vals]
    idx = np.ones_like(vals[0], dtype=bool)
    for vv in vals:
        lo, hi = np.percentile(vv, [2, 98])
        idx = idx & (lo < vv) & (vv < hi)
        
    vals = [vv[idx] for vv in vals]
    corner, _ = kale.corner(vals)
    corner.fig.text(0.5, 0.5, label, va='center', ha='center')
    
plt.show()


In [ ]:
fig, axes = plt.subplots(figsize=[10, 5], ncols=2, nrows=2, sharex='col')
nbins = [5, 10, 15, 0]

fit_keys = ['fit_lamp', 'fit_plaw']

for (ii, jj), ax in np.ndenumerate(axes):
    file = files[ii]
    ax.set(xscale='linear', yscale='linear')
    ax.grid(True, alpha=0.25)
    temp = file.parts[-2].split("_")[0].split("-")[-1]
    ax.set_title(temp, fontsize=12)
    data = h5py.File(file, 'r')

    key = fit_keys[jj]
    temp = data[key][:, :, 1]
    ave = np.mean(temp, axis=-1)
    print(utils.stats(ave))
    # kale.dist1d(ave, ax=ax)
    kale.dist1d(temp.flatten(), ax=ax)
    
    
plt.show()


## Single File

In [ ]:
#

In [ ]:
# fname = (
#     "/Users/lzkelley/programs/nanograv/holodeck/output/"
#     "hard04b_2023-01-23_01_n1000_g100_s40_r50_f40/"
#     "sam-lib_hard04b_2023-01-23_01_n1000_g100_s40_r50_f40.hdf5"
# )
# fname = "/Users/lzkelley/Programs/nanograv/holodeck/output/hard04b_2023-01-23_01_n1000_g100_s40_r50_f40/sam-lib_hard04b_2023-01-23_01_n1000_g100_s40_r50_f40.hdf5"

# fname = "/Users/lzkelley/programs/nanograv/holodeck/output/eccen-02_2023-01-31_02_n100_s30_r100_f20/sam_lib.hdf5"
FNAME = "/Users/lzkelley/programs/nanograv/holodeck/output/big-circ-01_2023-02-02_01_n2000_s40_r100_f40/sam_lib.hdf5"

print(FNAME)
data = h5py.File(FNAME, 'r')
print("Keys:", data.keys())
for kk, vv in data.items():
    try:
        print("\t", kk, vv.shape)
    except AttributeError:
        continue
    
print("\nAttributes:")
for kk, vv in data.attrs.items():
    print("\t", kk, vv)
    
# print("\nParameters:")
# for kk, vv in data['parameters'].items():
#     print("\t", kk, vv[:])


# Plot Strain Amplitude and Spectra

In [ ]:
fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax(xlabel='Frequency $[\\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)

if False:
    # (nsamples, nfreqs, nreals)
    yy = data['gwb'][:]
    # ==> (nfreqs, nsamples, nreals)
    yy = np.moveaxis(yy, 1, 0)
    yy = np.median(yy, axis=-1)
    yy = yy.reshape(yy.shape[0], -1)
    ax.set(ylim=[1e-16, 3e-14])
    ax.plot(xx, yy, alpha=0.1, color='k', lw=1)

    for amp in [1e-15, 2e-15, 3e-15]:
        plot._draw_plaw(ax, xx, amp, 1.0, color='r', alpha=1.0, zorder=100)

if True:
    df = np.fabs(xx - 1.0)
    ff = np.argmin(df)
    title = f"Characteristic Strain at ${xx[ff]:.2f} \, \\mathrm{{ yr}}^{{-1}}$"
    ax.set(xscale='linear', yscale='linear', xlim=[-17, -13], title=title)
    yy = data['gwb'][:, ff, :].flatten()
    yy = np.log10(yy[yy > 0.0])
    kale.dist1d(yy, ax=ax, density=True, carpet=True)

plt.show()

In [ ]:
num_label = 5
# data['gwb'].shape

NUM = 60
# percs = [25, 75]
percs = [10, 90]

fobs = data['fobs'][:]
xx = fobs * YR
fig, ax = plot.figax(xlabel='Frequency $[\\mathrm{yr}^{-1}]$', ylabel='Characteristic Strain')
plot._twin_hz(ax)
ax.set(ylim=[1e-16, 3e-14])

# (nsamples, nfreqs, nreals)
yy = data['gwb'][:]

nsamp = yy.shape[0]
choose = np.min([NUM, nsamp])
sel = np.random.choice(nsamp, choose, replace=False)
yy = yy[sel]

# ==> (nfreqs, nsamples, nreals)
yy = np.moveaxis(yy, 1, 0)

med = np.median(yy, axis=-1)

for ii in range(NUM):
    cc, = ax.plot(xx, med[:, ii], alpha=0.5, lw=1) # , color='b')
    cc = cc.get_color()

    conf = np.percentile(yy[:, ii, :], percs, axis=-1)
    ax.fill_between(xx, *conf, alpha=0.15, color=cc, lw=1)

    zz = yy[:, ii, :5]
    zz = zz.reshape(zz.shape[0], -1)
    ax.plot(xx, zz, alpha=0.25, color=cc, lw=1)

for amp in [1e-15, 2e-15, 3e-15]:
    hh = plot._draw_plaw(ax, xx, amp, 1.0, color='0.95', alpha=0.70, zorder=99, lw=2.0, ls='-')
    # zplot.label_line(ax, hh[0], f"{zplot.scientific_notation(amp)}", x=0.85, dy=0.01, alpha=0.65, fontsize=10)
    plot._draw_plaw(ax, xx, amp, 1.0, color='r', alpha=0.75, zorder=100, lw=1.0)

plt.show()

# Plot parameter space (corner plot)

In [ ]:
spars = data['sample_params'][:].copy()
names = [dd.decode() for dd in data.attrs['param_names']]
print(names, spars.shape)
idx = names.index('mmb_amp')
print(idx, names[idx])
spars[:, idx] = np.log10(spars[:, idx])
fig = plt.figure()
corner, _ = kale.corner(spars.T, dist2d=dict(contour=False), dist1d=dict(probability=True, density=False, hist=True), labels=names)
fig = corner.fig
fig.savefig('temp_params.png')
plt.show()

# Plot spectra versus different parameters

In [ ]:
percs = [25, 75]
xx = data['fobs'][:] * YR
gwb = data['gwb'][:]
spars = data['sample_params'][:]
print(gwb.shape, spars.shape)
# print(f"{len(data.attrs['param_names'])=}")

spars = spars.T
for ii, pname in enumerate(data.attrs['param_names']):
    pname = pname.decode()
    fig, ax = plot.figax(
        figsize=[6, 4], ylim=[0.5e-16, 3e-13], left=0.13, bottom=0.13, right=0.98, top=0.88,
        xlabel="Frequency [1/yr]", ylabel="Characteristic Strain",
    )
    plot._twin_hz(ax)

    vals = spars[ii].copy()
    sort_idx = np.argsort(vals)
    sv = vals[sort_idx]
    gwb = data['gwb'][:].copy()[sort_idx]

    num = gwb.shape[0]
    colors = mpl.cm.get_cmap('Spectral')(np.linspace(0.05, 0.95, num))

    for jj, pv in enumerate(sv):
        # Make sure there are only `num_label` labeled lines on the plot
        label = (jj % (num//num_label) == 0) or (num <= num_label) or (jj == num - 1)
        cc = colors[jj]

        yy = gwb[jj]
        med = np.median(yy, axis=-1)
        conf = np.percentile(yy, percs, axis=-1)
        label = f"{pv:.3e}" if label else ""
        ax.plot(xx, med, color=colors[jj], label=label, lw=1.0)
        ax.fill_between(xx, *conf, color=colors[jj], alpha=0.2, lw=0.5)
        
    ax.legend(title=pname, fontsize=8, loc='upper right')
    output_path = Path(holo._PATH_OUTPUT, f'eccen_lib_{pname}.png')
    fig.savefig(output_path, dpi=300)
    
plt.show()

# Look for NaN values in spectra --- Big_Circ01

In [ ]:
data = h5py.File(FNAME, 'r')
# (Samples, freqs, reals)
gwb = data['gwb']
print(gwb.shape)
bads = ~np.isfinite(gwb)
print(f"non-finite GWB values: {utils.frac_str(bads)}")

bads = np.any(bads, axis=(1, 2))
print(f"Parameters with any non-finite values (over: freqs, reals): {utils.frac_str(bads)}")

In [ ]:
SHAPE = 10

hard_time = 2.0
hard_rchar = 1.25
gamma_inner = -1.3
gamma_outer = 2.5
gsmf_phi0 = -2.2
gsmf_phiz = +0.4
gsmf_alpha0 = -2.3
gpf_malpha = -0.9
gpf_zbeta = 1.5
gpf_qgamma = -0.8
gmt_malpha = -0.15
gmt_zbeta = -0.5
gmt_qgamma = +0.75
mmb_amp = 9.75
mmb_plaw = 0.75

mmb_amp = (10.0 ** mmb_amp) * MSOL
hard_time = (10.0 ** hard_time) * GYR
hard_rchar = (10.0 ** hard_rchar) * PC

gsmf = holo.sam.GSMF_Schechter(phi0=gsmf_phi0, phiz=gsmf_phiz, alpha0=gsmf_alpha0)
gpf = holo.sam.GPF_Power_Law(malpha=gpf_malpha, qgamma=gpf_qgamma, zbeta=gpf_zbeta)
gmt = holo.sam.GMT_Power_Law(malpha=gmt_malpha, qgamma=gmt_qgamma, zbeta=gmt_zbeta)
mmbulge = holo.relations.MMBulge_KH2013(mamp=mmb_amp, mplaw=mmb_plaw)

sam = holo.sam.Semi_Analytic_Model(
    gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge,
    shape=SHAPE
)
hard = holo.hardening.Fixed_Time.from_sam(
    sam, hard_time, rchar=hard_rchar, gamma_sc=gamma_inner, gamma_df=gamma_outer,
    exact=True, progress=False
)

In [ ]:
pta_dur = 16.03
nfreqs = 40
REALS = 100
hifr = nfreqs/pta_dur
pta_cad = 1.0 / (2 * hifr)

fobs_cents = holo.utils.nyquist_freqs(pta_dur, pta_cad)
fobs_edges = holo.utils.nyquist_freqs_edges(pta_dur, pta_cad)
gwb = sam.gwb(fobs_edges, realize=REALS, hard=hard, zero_coalesced=False, zero_stalled=False)

In [ ]:
# (Samples, Dims)
pars = data['sample_params']
# (D,)
names = data.attrs['param_names']
for pp, pname in enumerate(names):
    fig, ax = plt.subplots()
    kale.dist1d(pars[:, pp], ax=ax)
    kale.dist1d(pars[bads, pp], ax=ax, color='r')
    ax.set_title(pname.decode(), fontsize=10)
    
plt.show()

In [ ]:
pars = data['sample_params']
names = data.attrs['param_names']
kale.corner(pars[bads].T, labels=names)
# for pp, pname in enumerate(data.attrs['param_names']):
#     print(pp, pname)
    
plt.show()